In [1]:
import os
import glob
import cv2 as cv
import numpy as np
import pandas as pd
from path import Path
from tqdm import tqdm
import nibabel as nib
import pydicom as dicom
import tensorflow as tf
from keras import layers
from pydicom import dcmread
from tensorflow import keras
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import requests
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import StratifiedKFold
from pydicom.pixel_data_handlers.util import apply_voi_lut
from keras_preprocessing.image import load_img, img_to_array

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train_b = pd.read_csv("train_bounding_boxes.csv")
submission = pd.read_csv("sample_submission.csv")

In [3]:
train

,StudyInstanceUID,patient_overall,C1,C2,C3,C4,C5,C6,C7
0,1.2.826.0.1.3680043.6200,1,1,1,0,0,0,0,0
1,1.2.826.0.1.3680043.27262,1,0,1,0,0,0,0,0
2,1.2.826.0.1.3680043.21561,1,0,1,0,0,0,0,0
3,1.2.826.0.1.3680043.12351,0,0,0,0,0,0,0,0
4,1.2.826.0.1.3680043.1363,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
2014,1.2.826.0.1.3680043.21684,1,0,1,0,0,0,1,1
2015,1.2.826.0.1.3680043.4786,1,0,0,0,0,0,0,1
2016,1.2.826.0.1.3680043.14341,0,0,0,0,0,0,0,0
2017,1.2.826.0.1.3680043.12053,0,0,0,0,0,0,0,0


In [4]:
test

,row_id,StudyInstanceUID,prediction_type
0,1.2.826.0.1.3680043.10197_C1,1.2.826.0.1.3680043.10197,C1
1,1.2.826.0.1.3680043.10454_C1,1.2.826.0.1.3680043.10454,C1
2,1.2.826.0.1.3680043.10690_C1,1.2.826.0.1.3680043.10690,C1


In [5]:
train_b

,StudyInstanceUID,x,y,width,height,slice_number
0,1.2.826.0.1.3680043.10051,219.27715,216.71419,17.30440,20.38517,133
1,1.2.826.0.1.3680043.10051,221.56460,216.71419,17.87844,25.24362,134
2,1.2.826.0.1.3680043.10051,216.82151,221.62546,27.00959,26.37454,135
3,1.2.826.0.1.3680043.10051,214.49455,215.48637,27.92726,37.51363,136
4,1.2.826.0.1.3680043.10051,214.00000,215.48637,27.00000,43.51363,137
...,...,...,...,...,...,...
7212,1.2.826.0.1.3680043.9940,297.23186,115.53983,85.18228,66.52623,140
7213,1.2.826.0.1.3680043.9940,298.00000,117.00000,86.00000,61.00000,141
7214,1.2.826.0.1.3680043.9940,298.00000,119.00000,87.00000,58.00000,142
7215,1.2.826.0.1.3680043.9940,299.00000,120.00000,89.00000,56.00000,143


In [6]:
submission

,row_id,fractured
0,1.2.826.0.1.3680043.10197_C1,0.5
1,1.2.826.0.1.3680043.10454_C1,0.5
2,1.2.826.0.1.3680043.10690_C1,0.5


In [8]:
def listDir(dir):
    FileName = os.listdir(dir)
    return len(FileName)

In [9]:
# for i in range(len(train)):
#     dir = r"D:\\Projects\\Kaggle\\RSNA\\train_images\\" + train['StudyInstanceUID'][i]
#     count = listDir(dir)
#     print(count)
#     for j in range(1,count):
#         sec = str(j) +".dcm"
#         image_path= os.path.abspath(os.path.join(dir,sec))
#         image_path = r"D:\\Projects\\Kaggle\\RSNA\\train_images\\"+train['StudyInstanceUID'][i]+'\\'+ 
#         ds = dicom.dcmread(image_path)
#         ds.PhotometricInterpretation = 'YBR_FULL'
#         pixel_array_numpy = ds.pixel_array
#         image_format = '.jpg' 
#         image_path = image_path.replace('.dcm', image_format)
#         cv2.imwrite(image_path, pixel_array_numpy)
#         print(count,j)
#         os.remove(image_path)


In [7]:
train.sort_values('StudyInstanceUID',inplace=True,axis = 0)
train_b.sort_values('StudyInstanceUID',inplace=True,axis = 0)
train.reset_index(inplace=True,drop= True)
train_b.reset_index(inplace=True,drop= True)

In [8]:
patients = sorted(os.listdir(r"D:\\Projects\\Kaggle\\RSNA\\train_images\\"))

In [9]:
patients[:5]

['1.2.826.0.1.3680043.10001',
 '1.2.826.0.1.3680043.10005',
 '1.2.826.0.1.3680043.10014',
 '1.2.826.0.1.3680043.10016',
 '1.2.826.0.1.3680043.10032']

In [10]:
ds = dcmread(r"D:\\Projects\\Kaggle\\RSNA\\train_images\\"+str('1.2.826.0.1.3680043.10001')+"\\1.dcm")
print(f"Image size: {ds.Rows} x {ds.Columns}")

Image size: 512 x 512


In [11]:
train_b.head()

,StudyInstanceUID,x,y,width,height,slice_number
0,1.2.826.0.1.3680043.10051,219.27715,216.71419,17.30440,20.38517,133
1,1.2.826.0.1.3680043.10051,191.22182,221.43460,61.44000,60.63086,147
2,1.2.826.0.1.3680043.10051,195.00000,221.00000,51.00000,58.00000,146
3,1.2.826.0.1.3680043.10051,199.60000,220.62545,40.02908,56.78546,145
4,1.2.826.0.1.3680043.10051,201.00000,219.00000,41.00000,58.00000,144


In [12]:
train_b['startX']=0.0
train_b['startY']=0.0
train_b['endX']=0.0
train_b['endY']=0.0
for i in range(len(train_b)):
    # calculate top,left for box
    train_b['startX'][i] = train_b['x'][i] - (train_b['width'][i]/2)
    train_b['startY'][i] = train_b['y'][i] + (train_b['height'][i]/2)
    # calculate right,bottom for box
    train_b['endX'][i] = train_b['x'][i] + (train_b['width'][i]/2)
    train_b['endY'][i] = train_b['y'][i] - (train_b['height'][i]/2)


C:\Users\maxme\AppData\Local\Temp\ipykernel_20200\2287673146.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_b['startX'][i] = train_b['x'][i] - (train_b['width'][i]/2)
C:\Users\maxme\AppData\Local\Temp\ipykernel_20200\2287673146.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_b['startY'][i] = train_b['y'][i] + (train_b['height'][i]/2)
C:\Users\maxme\AppData\Local\Temp\ipykernel_20200\2287673146.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

In [ ]:
def load_dicom(path):
    img=dicom.dcmread(path)
    data=img.pixel_array
    data=data-np.min(data)
    if np.max(data) != 0:
        data=data/np.max(data)
    data=(data*255).astype(np.uint8)
    return data

In [ ]:
# image_file = glob.glob(r"D:\\Projects\\Kaggle\\RSNA\\train_images\\"+str('1.2.826.0.1.3680043.10001')+"\\*.dcm")
# plt.figure(figsize=(20, 20))

# for i in range(28):
#     ax = plt.subplot(7, 7, i + 1)
#     # specify your dcm image path
#     image_path = image_file[i]

#     image = load_dicom(image_path)

#     plt.axis('off')
    
#     plt.imshow(image)

In [13]:
train_dir=r"D:\\Projects\\Kaggle\\RSNA\\train_images\\"
test_dir = r"D:\\Projects\\Kaggle\\RSNA\\test_images\\"

In [101]:
data = []
dir  = r"D:\\Projects\\Kaggle\\RSNA\\train_images\\" + train['StudyInstanceUID'][i]
sec = str(train_b['slice_number'][0]) +".dcm"
image_path= os.path.abspath(os.path.join(dir,sec))
ds = dicom.dcmread(image_path)
# ds.PhotometricInterpretation = 'YBR_FULL'
pixel_array_numpy = ds.pixel_array
data.append(pixel_array_numpy)
dir  = r"D:\\Projects\\Kaggle\\RSNA\\train_images\\" + train['StudyInstanceUID'][i]
sec = str(train_b['slice_number'][1]) +".dcm"
image_path= os.path.abspath(os.path.join(dir,sec))
ds = dicom.dcmread(image_path)
# ds.PhotometricInterpretation = 'YBR_FULL'
pixel_array_numpy = ds.pixel_array
data.append(pixel_array_numpy)


In [103]:
x_train = np.asarray(data).astype(np.float32)

In [108]:
x_train.shape

(2, 1, 512, 512)

In [14]:
data =[]
targets = []
filenames = []
trainlabel = []
for i in range(len(train)):
    dir  = r"D:\\Projects\\Kaggle\\RSNA\\train_images\\" + train['StudyInstanceUID'][i]
    for j in range(len(train_b)):
        if train['StudyInstanceUID'][i] == train_b['StudyInstanceUID'][j]:
            sec = str(train_b['slice_number'][j]) +".dcm"
            image_path= os.path.abspath(os.path.join(dir,sec))
            ds = dicom.dcmread(image_path)
            pixel_array_numpy = ds.pixel_array
            data.append(pixel_array_numpy)
            targets.append([train_b['startX'][j], train_b['startY'][j], train_b['endX'][j], train_b['endY'][j]])
            filenames.append(train_b['StudyInstanceUID'][j])
            cur_label = []
            cur_label.append(train.loc[i,'C1'])
            cur_label.append(train.loc[i,'C2'])
            cur_label.append(train.loc[i,'C3'])
            cur_label.append(train.loc[i,'C4'])
            cur_label.append(train.loc[i,'C5'])
            cur_label.append(train.loc[i,'C6'])
            cur_label.append(train.loc[i,'C7'])
            trainlabel += [cur_label]

In [ ]:
data = []
dir  = r"D:\\Projects\\Kaggle\\RSNA\\train_images\\" + train['StudyInstanceUID'][i]
sec = str(train_b['slice_number'][0]) +".dcm"
image_path= os.path.abspath(os.path.join(dir,sec))
print(image_path)
image = cv.imread(image_path)
print(image)
# cv.resize(image, (64 , 64))
# image = img_to_array(image)
# data.append(image)

# dir  = r"D:\\Projects\\Kaggle\\RSNA\\train_images\\" + train['StudyInstanceUID'][i]
# sec = str(train_b['slice_number'][1]) +".dcm"
# image_path= os.path.abspath(os.path.join(dir,sec))
# image = cv.imread(image_path)
# cv.resize(image, (64 , 64))
# image = img_to_array(image)
# data.append(image)



In [26]:
data =[]
targets = []
filenames = []
trainlabel = []
for i in range(len(train)):
    dir  = r"D:\\Projects\\Kaggle\\RSNA\\train_images\\" + train['StudyInstanceUID'][i]
    for j in range(len(train_b)):
        if train['StudyInstanceUID'][i] == train_b['StudyInstanceUID'][j]:
            sec = str(train_b['slice_number'][j]) +".dcm"
            image_path= os.path.abspath(os.path.join(dir,sec))
            ds = dicom.dcmread(image_path)
            pixel_array_numpy = ds.pixel_array
            data.append(pixel_array_numpy)
            targets.append([train_b['startX'][j], train_b['startY'][j], train_b['endX'][j], train_b['endY'][j]])
            filenames.append(train_b['StudyInstanceUID'][j])
            cur_label = []
            cur_label.append(train.loc[i,'C1'])
            cur_label.append(train.loc[i,'C2'])
            cur_label.append(train.loc[i,'C3'])
            cur_label.append(train.loc[i,'C4'])
            cur_label.append(train.loc[i,'C5'])
            cur_label.append(train.loc[i,'C6'])
            cur_label.append(train.loc[i,'C7'])
            trainlabel += [cur_label]

In [15]:
data

[array([[-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        ...,
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000]], dtype=int16),
 array([[-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        ...,
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000]], dtype=int16),
 array([[-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        ...,
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -20

In [16]:
targets

[[210.62495, 226.90677499999998, 227.92935, 206.52160500000002],
 [160.50182, 251.75002999999998, 221.94182, 191.11917],
 [169.5, 250.0, 220.5, 192.0],
 [179.58546, 249.01818, 219.61453999999998, 192.23272],
 [180.5, 248.0, 221.5, 190.0],
 [182.0, 248.03921499999998, 224.0, 185.96078500000002],
 [183.73599000000002, 246.48496, 226.63491, 183.05131999999998],
 [188.49865, 248.382985, 211.82719000000003, 216.459735],
 [182.5, 243.43671, 227.5, 184.31013000000002],
 [190.96934499999998, 240.36007999999998, 227.03065500000002, 188.79174],
 [200.996365, 240.677775, 226.130915, 192.750605],
 [200.5, 237.24318499999998, 227.5, 193.729555],
 [200.53092, 234.24318499999998, 228.45818, 196.729555],
 [203.316715, 234.81273000000002, 230.326305, 208.43819],
 [212.62538, 229.336, 230.50382000000002, 204.09238],
 [180.51637, 241.10544499999997, 227.99273, 183.389095],
 [232.75173, 266.24312, 267.17931, 223.68792],
 [232.41685999999999, 261.86113, 267.94678, 228.06991],
 [227.17163, 262.642285, 267.1

In [16]:
data = np.array(data)
print(data.shape)

(7217,)


C:\Users\maxme\AppData\Local\Temp\ipykernel_7284\3332820839.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  data = np.array(data)


In [19]:
targets= np.array(targets)

In [21]:
targets.shape

(7217, 4)

In [ ]:
trainset

In [ ]:
from tensorflow.keras.applications import VGG16

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,Input, Conv3D, MaxPooling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda,Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
kernel_initializer =  'he_uniform'
def get_model():
   inp = keras.layers.Input((None, None ,1))
      #Contraction path
   x = Conv2D(3, 3, padding = 'SAME')(inp)
   x = Xception(include_top=False)(x)
   x = keras.layers.GlobalAveragePooling2D()(x)
   out = keras.layers.Dense(7, 'softmax')(x)
   model = keras.models.Model(inp, out)
  
   model.compile(loss="categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001))
  
    #compile model outside of this function to make it flexible. 

   return model

In [ ]:
model = get_model()  
model.compile(loss="categorical_crossentropy", optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001))

In [ ]:
submission

In [ ]:
submission.to_csv('target.csv', index = 0)

# THE END!

In [ ]:
for i in range(len(train)):
    dir  = r"D:\\Projects\\Kaggle\\RSNA\\train_images\\" + train['StudyInstanceUID'][i]
    for j in range(len(train_b)):
        if train['StudyInstanceUID'][i] == train_b['StudyInstanceUID'][j]:
            sec = str(train_b['slice_number'][j]) +".dcm"
            image_path= os.path.abspath(os.path.join(dir,sec))
            image = load_dicom(image_path)
            cv.resize(image, (64 , 64))
            image = img_to_array(image)
            data.append(image)
            targets.append([train_b['startX'][j], train_b['startY'][j], train_b['endX'][j], train_b['endY'][j]])
            filenames.append(train_b['StudyInstanceUID'][j])
            cur_label = []
            cur_label.append(train.loc[i,'C1'])
            cur_label.append(train.loc[i,'C2'])
            cur_label.append(train.loc[i,'C3'])
            cur_label.append(train.loc[i,'C4'])
            cur_label.append(train.loc[i,'C5'])
            cur_label.append(train.loc[i,'C6'])
            cur_label.append(train.loc[i,'C7'])
            trainlabel += [cur_label]

In [ ]:
train_data = np.array(data[0], dtype="float32") / 255.0
y = np.array(targets, dtype="float32")

In [ ]:
train_data.shape

In [ ]:
y

In [ ]:
train_b

In [ ]:
testset=[]
testid=[]

In [ ]:
test

In [ ]:
# for i in range(len(test)):
#     dir = r"D:\\Projects\\Kaggle\\RSNA\\test_images\\" + test['StudyInstanceUID'][i]
#     for im in os.listdir(dir):
#         dc = dicom.read_file(os.path.join(dir,im))
#         img=load_dicom(os.path.join(dir,im))
#         img=cv.resize(img,(64,64))
#         image=img_to_array(img)
#         image=image/255.0
#         testset.append(image)
#         testid.append(test['StudyInstanceUID'][i])

In [ ]:
for i in (range(len(test))):        
    if type(test) is list: idt = test[i]
    else: idt = test['StudyInstanceUID'].iloc[i]
    path = os.path.join(r"D:\\Projects\\Kaggle\\RSNA\\test_images\\", idt)
    if os.path.exists(path):
        for im in os.listdir(path):
            dc = dicom.read_file(os.path.join(path,im))
            if dc.file_meta.TransferSyntaxUID.name =='JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':
                continue
            img=load_dicom(os.path.join(path,im))
            img=cv.resize(img,(64,64))
            image=img_to_array(img)
            image=image/255.0
            testset+=[image]
            testid+=[idt]

In [19]:
import tensorflow
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanIoU
kernel_initializer =  'he_uniform'

In [20]:
def get_model():
    inputs = Input((None, None ,1))
    #s = Lambda(lambda x: x / 255)(inputs)   #No need for this if we normalize our inputs beforehand
    s = inputs

    #Contraction path
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(s)
    c1 = Dropout(0.1)(c1)
    c1 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p1)
    c2 = Dropout(0.1)(c2)
    c2 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
     
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p2)
    c3 = Dropout(0.2)(c3)
    c3 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
     
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p3)
    c4 = Dropout(0.2)(c4)
    c4 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c4)
    p4 = MaxPooling2D(pool_size=(2, 2))(c4)
     
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(p4)
    c5 = Dropout(0.3)(c5)
    c5 = Conv2D(256, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c5)
    
    #Expansive path 
    u6 = Conv2DTranspose(128, (2, 2), strides=2, padding='same')(c5)
    u6 = concatenate([u6, c4])
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u6)
    c6 = Dropout(0.2)(c6)
    c6 = Conv2D(128, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c6)
     
    u7 = Conv2DTranspose(64, (2, 2), strides=2, padding='same')(c6)
    u7 = concatenate([u7, c3])
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u7)
    c7 = Dropout(0.2)(c7)
    c7 = Conv2D(64, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c7)
     
    u8 = Conv2DTranspose(32, (2, 2), strides=2, padding='same')(c7)
    u8 = concatenate([u8, c2])
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u8)
    c8 = Dropout(0.1)(c8)
    c8 = Conv2D(32, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c8)
     
    u9 = Conv2DTranspose(16, (2, 2), strides=2, padding='same')(c8)
    u9 = concatenate([u9, c1])
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(u9)
    c9 = Dropout(0.1)(c9)
    c9 = Conv2D(16, (3, 3), activation='relu', kernel_initializer=kernel_initializer, padding='same')(c9)
  
    outputs = Conv2D(7, (1, 1), activation='softmax')(c9)
     
    model = Model(inputs=[inputs], outputs=[outputs])
    #compile model outside of this function to make it flexible. 

    return model

In [21]:
model = get_model()
model.compile(optimizer = "Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [77]:
data

[array([[-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        ...,
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000]], dtype=int16),
 array([[-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        ...,
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000]], dtype=int16),
 array([[-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        ...,
        [-2000, -2000, -2000, ..., -2000, -2000, -2000],
        [-2000, -20

In [75]:
y = np.array(targets)

In [76]:
y.shape

(7217, 4)

In [71]:
from sklearn.model_selection import train_test_split
split = train_test_split(data, y,trainlabel, test_size=0.10,
	random_state=42)
(trainImages, valImages) = split[:2]
(trainTargets, valTargets) = split[2:4]

(trainFilenames, valFilenames) = split[4:]


In [74]:
H = model.fit(
	data, targets,
	validation_data=0.1,
	batch_size=64,
	epochs=3,
	verbose=1)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {'(<class \'list\'> containing values of types {"<class \'numpy.float64\'>"})'})

In [ ]:
for i in range(len(train_data)):
    if train_data.shape[0]==32:
        print(train_data.shape[0])

In [ ]:
train_data.shape

In [ ]:
history = model.fit(
    train_data,
    epochs=10,
    verbose=1,
    validation_data=0.1,

)

In [ ]:
vgg = VGG16(weights="imagenet", include_top=False,
input_tensor=Input(shape=(None, None ,1)))
# freeze all VGG layers so they will *not* be updated during the
# training process
vgg.trainable = False
# flatten the max-pooling output of VGG
flatten = vgg.output
flatten = Flatten()(flatten)
# construct a fully-connected layer header to output the predicted
# bounding box coordinates
bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid")(bboxHead)
# construct the model we will fine-tune for bounding box regression
model = Model(inputs=vgg.input, outputs=bboxHead)

In [ ]:
opt = Adam()
model.compile(loss="mse", optimizer=opt)
print(model.summary())

In [ ]:
y=np.array(targets)

In [ ]:
y.shape

In [ ]:


H = model.fit(
	train_data,
	validation_data=0.1,
	batch_size=32,
	epochs=10,
	verbose=1)

In [ ]:
train_data